## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![jupyter](./fig/4_1.jpg)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：  
$问题1证明：若最小生成树MST不是瓶颈生成树BST，则将MST中的最大权重边替换为瓶颈树的对应边，形成的新最小生成树比原MST权重和小$  
$与原最小生成树MST是MST矛盾，故而每一个MST都是BST$  

$问题2思路：过滤出所有权值不超过b的边，构成子图G'$  
$检查G'是否连通。若连通，则存在满足条件的BST$  
$时间复杂度：遍历一遍边O(E)，检查是否连通O(V)，总时间复杂度O(E+V)$

$问题3思路：二分查找问题2的最小b$  
$时间复杂度：二分查找最多进行 O(logW) 次（W 为最大边权），每次检查连通性耗时 O(V+E),总时间复杂度为O((V+E)logW)$

In [5]:
# add your code here
def has_bst(G, b):
    vertices, edges = G
    adj = [[] for _ in vertices]
    for u, v, w in edges:
        if w <= b:
            adj[u].append(v)
            adj[v].append(u)
    visited = [False] * len(vertices)
    stack = [0]
    visited[0] = True
    count = 1
    while stack:
        u = stack.pop()
        for v in adj[u]:
            if not visited[v]:
                visited[v] = True
                count += 1
                stack.append(v)
    return count == len(vertices)
# 示例图
vertices = [0, 1, 2]
edges = [(0, 1, 3), (1, 2, 4), (0, 2, 5)]
G = (vertices, edges)

# 测试输出
print(has_bst(G, 4))  # 输出: True（存在满足条件的BST）
print(has_bst(G, 3))  # 输出: False（不存在满足条件的BST）

def find_bottleneck_tree(G):
    vertices, edges = G
    edge_weights = [w for _, _, w in edges]
    if not edge_weights:
        return []
    left = min(edge_weights)
    right = max(edge_weights)
    answer = right

    # 二分查找确定最小的 b
    while left <= right:
        mid = (left + right) // 2
        if has_bst(G, mid):
            answer = mid
            right = mid - 1
        else:
            left = mid + 1

    # 构造生成树（BFS）
    adj = [[] for _ in vertices]
    for u, v, w in edges:
        if w <= answer:
            adj[u].append(v)
            adj[v].append(u)
    parent = [-1] * len(vertices)
    visited = [False] * len(vertices)
    queue = [0]
    visited[0] = True
    tree_edges = []
    while queue:
        u = queue.pop(0)
        for v in adj[u]:
            if not visited[v]:
                visited[v] = True
                parent[v] = u
                tree_edges.append((u, v))
                queue.append(v)
    return tree_edges

# 示例图
vertices = [0, 1, 2, 3]
edges = [(0, 1, 2), (0, 2, 3), (1, 2, 4), (1, 3, 5), (2, 3, 6)]
G = (vertices, edges)

# 调用函数并输出结果
tree = find_bottleneck_tree(G)
print("Bottleneck Spanning Tree Edges:", tree)
# 可能的输出: [(0, 1), (0, 2), (1, 3)]
# algorithm of the liear time complexity :O(E+V)、O((E+V)logW)E为边数,V为节点数,W为边最大权值

True
False
Bottleneck Spanning Tree Edges: [(0, 1), (0, 2), (1, 3)]


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea：  
$使用Dijkstra算法计算原图中所有顶点对之间的最短路径，得到一个距离矩阵$  
$预处理一个矩阵，用于存储每条可能的候选边(u, v)能带来的最大距离减少量$  
$遍历所有候选边，利用预处理得到的矩阵快速计算每条边的最大收益，并选择收益最大的边$  
$预处理最短路径：O(n(m + n log n))，其中n是顶点数，m是边数，计算矩阵M和A：O(n³)，评估候选边：O(k)，其中k是候选边数量$

In [8]:
# add your code here
import heapq

def add_new_road(G, E_prime):
    V = G['vertices']
    E = G['edges']
    l = G['lengths']
    n = len(V)
    INF = float('inf')
    
    # Step 1: Precompute all pairs shortest paths using Dijkstra
    d = [[INF] * n for _ in range(n)]
    for s in range(n):
        d[s][s] = 0
        heap = []
        heapq.heappush(heap, (0, s))
        visited = [False] * n
        while heap:
            dist_u, u = heapq.heappop(heap)
            if visited[u]:
                continue
            visited[u] = True
            for (v, length) in E[u]:
                if d[s][v] > dist_u + length:
                    d[s][v] = dist_u + length
                    heapq.heappush(heap, (d[s][v], v))
    
    # Step 2a: Precompute M[s][v] = max_t (d[s][t] - d[v][t])
    M = [[-INF for _ in range(n)] for _ in range(n)]
    for v in range(n):
        for s in range(n):
            max_val = -INF
            for t in range(n):
                if d[s][t] != INF and d[v][t] != INF:
                    current = d[s][t] - d[v][t]
                    if current > max_val:
                        max_val = current
            M[s][v] = max_val
    
    # Step 2b: Precompute A[u][v] = max_s (M[s][v] - d[s][u])
    A = [[-INF for _ in range(n)] for _ in range(n)]
    for u in range(n):
        for v in range(n):
            max_benefit = -INF
            for s in range(n):
                if M[s][v] != -INF and d[s][u] != INF:
                    current = M[s][v] - d[s][u]
                    if current > max_benefit:
                        max_benefit = current
            A[u][v] = max_benefit
    
    # Step 3 & 4: Evaluate all candidate edges
    max_reduction = -1
    best_edge = None
    for (u, v, l_prime) in E_prime:
        if A[u][v] == -INF:
            reduction = 0
        else:
            reduction = max(0, A[u][v] - l_prime)
        if reduction > max_reduction:
            max_reduction = reduction
            best_edge = (u, v)
        elif reduction == max_reduction and best_edge is None:
            best_edge = (u, v)
    
    return best_edge, max_reduction

# 示例用法
# 示例图结构，这里用顶点索引表示
# 例如，顶点0到1有一条长度为2的边，顶点0到2有一条长度为3的边，等等。
G = {
    'vertices': [0, 1, 2],
    'edges': {
        0: [(1, 2), (2, 3)],
        1: [(2, 1)],
        2: []
    },
    'lengths': {(0,1):2, (0,2):3, (1,2):1}
}
# 候选边列表，格式为(u, v, l')
E_prime = [(0, 1, 1), (2, 0, 1)]
best_edge, reduction = add_new_road(G, E_prime)
print(f"Best edge to add: {best_edge}, distance reduction: {reduction}")
# algorithm of the liear time complexity :O(E+V)、O((E+V)logW)E为边数,V为节点数,W为边最大权值

Best edge to add: (0, 1), distance reduction: 1


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：  
$第一题思路：给定原网络 G=(V,E)(含顶点容量 c(v) 和边容量 c(e))，构造新网络 G'=(V',E') $  
$对每个顶点 v∈V，生成两个新顶点 v\_in和 v\_out，且 V'=\{v\_in,v\_out∣v∈V\}$  
$对每个 v∈V，添加边 (v\_in→v\_out) 到 E'，容量为顶点容量 c(v)$
$对原边 e=(u→v)∈E，将其替换为 (u\_out→v\_in)∈E'，容量保持 c(e)$
$需证明 G 和 G'的最大流值相等，且构造满足 ∣V'∣=O(∣V∣)、∣E'∣=O(∣E∣)$
$顶点容量约束等价性:在 G 中，进入顶点 v 的总流量受 c(v) 限制,在 G'中，所有进入 v\_in的流量必须经过边 (v\_in→v\_out)，其容量为 c(v)。因此，v\_in→v\_out的边天然限制了进入 v 的总流量，与原网络顶点容量约束一致$
$边容量保留:原边 e=(u→v) 的容量 c(e) 在 G'中直接保留于边 (u\_out→v\_in)$
$流守恒的保持:在 G'中，原顶点 v 的流守恒性由 v\_in和 v\_out的流守恒性共同保证：进入 v\_in的流量等于离开 v\_out的流量（因内部边 v\_in→v\_out的容量限制）$  
$原边 (u→v) 的流量守恒性由 u\_out→v\_in的容量和流守恒性直接继承$
$最大流等价性:对 G 中的任一可行流 f，在 G'中构造流 f'：$
$对原边 e=(u→v)，令 f'(u\_out→v\_in)=f(e)$
$对内部边 v\_in→v\_out，令 f'(v\_in→v\_out)=∑(e进入v)f(e)$
$G'中的任一可行流f'可还原为 G 中的可行流 f，且 ∣f∣=∣f'∣。因此，两网络最大流值相等$

$第二题：将网格顶点 (i,j) 拆分为入点 (i,j)\_in和出点 (i,j)\_out，中间边容量为 1（顶点不相交约束）$
$网格的无向边转换为两条有向边(如 (u\_out→v\_in) 和 (v\_out→u\_in))，容量为 1$
$添加源点 S 和汇点 T$
$源点 S 连接到所有起点 (x_k,y_k)\_in，容量为 1$  
$所有边界顶点的出点 (i,j)\_out连接到汇点 T，容量为 1$  
$使用 Dinic 算法计算从 S 到 T 的最大流。若最大流等于 m，则存在 m 条顶点不相交路径；否则不存在$
$时间复杂度:顶点数 V=O(n^2)，边数 E=O(n^2)（每个顶点拆分为 2 个，每条无向边拆分为 2 条有向边）$
$Dinic 算法在单位容量网络中的复杂度为 O(E\sqrt{V})=O(n^3)$  
$总时间复杂度为O(n^3)$